In [1]:
import os, random, time
import xgboost
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#from fbprophet import Prophet
from sklearn.linear_model import LinearRegression as LR
from sklearn.kernel_ridge import KernelRidge as KR
from sklearn.linear_model import Ridge, ElasticNet, BayesianRidge, Lars, Lasso
from sklearn.ensemble import RandomForestRegressor as RF

from sklearn.metrics import r2_score
from sklearn.gaussian_process import GaussianProcessRegressor as gpr
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel

import warnings

from bayes_opt import BayesianOptimization

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

In [2]:
y_inven = pd.read_csv('./E61D_Inven.csv')
x_sales = pd.read_csv('./E61D_Sale.csv')
x_product = pd.read_csv('./E61D_Product.csv')
d_set = pd.merge(pd.merge(y_inven, x_sales, how='left', on='Week'), x_product, how='left', on='Week')
d_set = d_set.drop(columns=['5xxx', '3xxC', '3xxT'])
d_set[d_set < -1000] = 0
#d_set_log = d_set.copy()
#d_set_log[d_set < 0] = 0
#d_set.to_csv('data_0506.csv', index=False)

rm_week = [201952, 202001, 202052, 202053]#, 202101]
rm_index = [i for i,s in enumerate(d_set['Week']) if s not in rm_week]
d_set1 = d_set.loc[rm_index]
d_set1 = d_set1.reset_index(drop=True)


In [3]:
tr_s_day, tr_e_day=202002, 202040
te_s_day, te_e_day=202040, 202051
x_dta = d_set1
target_key = 'Product'

tr_time = np.array([tr_s_day, tr_e_day])
te_time = np.array([te_s_day, te_e_day])
s_ind, e_ind = np.where(tr_time[0] == x_dta['Week'])[0][0], np.where(tr_time[1] == x_dta['Week'])[0][0]
te_s_ind, te_e_ind = np.where(te_time[0] == x_dta['Week'])[0][0], np.where(te_time[1] == x_dta['Week'])[0][0]

target_col = [s for s in x_dta.columns.values if target_key == s]    
tmp_x_col = [s for s in x_dta.columns.values if 'Prev_'+target_key in s]
tmp_x_col1 = [s for s in x_dta.columns.values if 'Var_'+target_key in s]
week_col = [s for s in x_dta.columns.values if 'Week' in s]

tmp_x_set1, tmp_x_set2 = x_dta[tmp_x_col], x_dta[tmp_x_col1]


In [4]:
warnings.filterwarnings(action='ignore', category=UserWarning)
param_bound = {'theta1' : (0, 10), 'g_alpha' : (0.0,0.99) , 'k_dot' : (0, 100), 'w_noise' : (0.0, 100), 'rbf_l' : (0, 10), 'col1' : (1,13), 'col2' : (1,13)}

def h_opt(theta1, g_alpha, k_dot, w_noise, rbf_l, col1, col2):
    
    col_k = round(col1)
    col_k1 = round(col2)
    #tmp_x_set1 = np.multiply(tmp_x_set1, np.tile([wt_mat], tmp_x_set1.shape[0]).reshape(tmp_x_set1.shape[0], -1)).copy()
    #tmp_x_set2 = np.multiply(tmp_x_set2, np.tile([wt_mat], tmp_x_set2.shape[0]).reshape(tmp_x_set2.shape[0], -1)).copy()
    if target_key == 'Product':
        tmp_x_set1_re = tmp_x_set1.iloc[:,:col_k]
        tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
        x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set1_re, tmp_x_set2_re], axis=1)
        train_col = list(tmp_x_col[:col_k]+tmp_x_col1[:col_k1])    
    elif target_key == 'Sale':
        tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
        x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set2_re], axis=1)
        train_col = list(tmp_x_col1[:col_k1])


    train_set = x_dta_re.iloc[s_ind:e_ind, :]
    test_set = x_dta_re.iloc[te_s_ind:(te_e_ind+1), :]#.copy()

    tr_x_set = train_set[train_col]
    tr_y_set = np.array(train_set[target_col]).reshape(-1,1)+1e-5
    te_x_set = test_set[train_col]
    te_y_set = np.array(test_set[target_col]).reshape(-1,1)+1e-5

    kernel =  theta1 * RBF(rbf_l) + WhiteKernel(w_noise)  + DotProduct(k_dot) ## RBF(2) + WhiteKernel(1)  + 
    gp = GaussianProcessRegressor(kernel=kernel, alpha=g_alpha, n_restarts_optimizer=3)

    gp.fit(tr_x_set, tr_y_set)
    train_pred, train_sigma = gp.predict(tr_x_set, return_std=True)
    test_pred, test_sigma = gp.predict(te_x_set, return_std=True)
    x_pred_ = np.concatenate((train_pred, test_pred)) 
    y_ = np.concatenate((tr_y_set, te_y_set)) 

    real_ = np.mean(np.ones_like(tr_y_set) - np.clip(np.abs(tr_y_set - train_pred) / tr_y_set , 0, 1)) * 100     
    fcst_ = np.mean(np.ones_like(te_y_set) - np.clip(np.abs(te_y_set - test_pred) / te_y_set , 0, 1)) * 100    
    return fcst_    

product_optimizer = BayesianOptimization(f=h_opt, pbounds=param_bound, verbose=2, random_state=1)
product_optimizer.maximize(init_points=40, n_iter=50)

|   iter    |  target   |   col1    |   col2    |  g_alpha  |   k_dot   |   rbf_l   |  theta1   |  w_noise  |
-------------------------------------------------------------------------------------------------------------
|  1        |  72.8     |  6.004    |  9.644    |  0.000113 |  30.23    |  1.468    |  0.9234   |  18.63    |
|  2        |  84.02    |  5.147    |  5.761    |  0.5334   |  41.92    |  6.852    |  2.045    |  87.81    |
|  3        |  71.24    |  1.329    |  9.046    |  0.4131   |  55.87    |  1.404    |  1.981    |  80.07    |
|  4        |  84.52    |  12.62    |  4.761    |  0.6854   |  87.64    |  8.946    |  0.8504   |  3.905    |
|  5        |  71.33    |  3.038    |  11.54    |  0.09736  |  42.11    |  9.579    |  5.332    |  69.19    |
|  6        |  73.61    |  4.786    |  9.238    |  0.8263   |  1.829    |  7.501    |  9.889    |  74.82    |
|  7        |  68.84    |  4.365    |  10.47    |  0.1022   |  44.79    |  9.086    |  2.936    |  28.78    |
|  8      

AttributeError: 'str' object has no attribute 'decode'

In [ ]:
#|   iter    |  target   |   col1    |   col2    |  g_alpha  |   k_dot   |   rbf_l   |  theta1   |  w_noise  |
###########  |  75.45    |  4.412    |  1.754    |  0.4299   |  83.78    |  2.45     |  7.428    |  95.67    |

tr_s_day, tr_e_day=202002, 202040
te_s_day, te_e_day=202040, 202051
x_dta = d_set1
target_key = 'Sale'
col_k, col_k1 = round(4.412), round(1.754)
g_alpha, k_dot, rbf_l, theta1, w_noise =  0.4299   ,  83.78    , 2.45     ,  7.428   ,  95.67

tr_time = np.array([tr_s_day, tr_e_day])
te_time = np.array([te_s_day, te_e_day])
s_ind, e_ind = np.where(tr_time[0] == x_dta['Week'])[0][0], np.where(tr_time[1] == x_dta['Week'])[0][0]
te_s_ind, te_e_ind = np.where(te_time[0] == x_dta['Week'])[0][0], np.where(te_time[1] == x_dta['Week'])[0][0]

target_col = [s for s in x_dta.columns.values if target_key == s]    
tmp_x_col = [s for s in x_dta.columns.values if 'Prev_'+target_key in s]
tmp_x_col1 = [s for s in x_dta.columns.values if 'Var_'+target_key in s]
week_col = [s for s in x_dta.columns.values if 'Week' in s]

tmp_x_set1, tmp_x_set2 = x_dta[tmp_x_col], x_dta[tmp_x_col1]

if target_key == 'Product':
    tmp_x_set1_re = tmp_x_set1.iloc[:,:col_k]
    tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
    x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set1_re, tmp_x_set2_re], axis=1)
    train_col = list(tmp_x_col[:col_k]+tmp_x_col1[:col_k1])    
elif target_key == 'Sale':
    tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
    x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set2_re], axis=1)
    train_col = list(tmp_x_col1[:col_k1])


train_set = x_dta_re.iloc[s_ind:e_ind, :]
test_set = x_dta_re.iloc[te_s_ind:(te_e_ind+1), :]#.copy()

tr_x_set = train_set[train_col]
tr_y_set = np.array(train_set[target_col]).reshape(-1,1)+1e-5
te_x_set = test_set[train_col]
te_y_set = np.array(test_set[target_col]).reshape(-1,1)+1e-5



kernel =  theta1 * RBF(rbf_l) + WhiteKernel(w_noise)  + DotProduct(k_dot) ## RBF(2) + WhiteKernel(1)  + 
gp = GaussianProcessRegressor(kernel=kernel, alpha=g_alpha, n_restarts_optimizer=3)
#tr_x_ = np.atleast_2d([np.arange(0, np.shape(tr_x_set)[0])]).T
gp.fit(tr_x_set, tr_y_set)
train_pred, train_sigma = gp.predict(tr_x_set, return_std=True)
test_pred, test_sigma = gp.predict(te_x_set, return_std=True)
x_pred_ = np.concatenate((train_pred, test_pred)) 
y_ = np.concatenate((tr_y_set, te_y_set)) 

real_ = np.mean(np.ones_like(tr_y_set) - np.clip(np.abs(tr_y_set - train_pred) / tr_y_set , 0, 1)) * 100     
fcst_ = np.mean(np.ones_like(te_y_set) - np.clip(np.abs(te_y_set - test_pred) / te_y_set , 0, 1)) * 100    
print(real_, fcst_)

plt.figure(figsize=(20,10))
plt.plot(y_, color='blue', marker='o')
plt.plot(x_pred_, color='red', linestyle='dashed', marker='o')
plt.axvline(len(tr_y_set)-1, c='k')

In [ ]:
#|   iter    |  target   |   col1    |   col2    |  g_alpha  |   k_dot   |   rbf_l   |  theta1   |  w_noise  |
###########  |  87.21    |  13.0     |  7.566    |  0.0      |  100.0    |  7.361    |  5.031    |  49.11    |

tr_s_day, tr_e_day=202002, 202040
te_s_day, te_e_day=202040, 202051
x_dta = d_set1
target_key = 'Product'
col_k, col_k1 = round(13), round(7.566)
g_alpha, k_dot, rbf_l, theta1, w_noise =   0.0 , 100.0   ,  7.361   ,  5.031   ,  49.11 

tr_time = np.array([tr_s_day, tr_e_day])
te_time = np.array([te_s_day, te_e_day])
s_ind, e_ind = np.where(tr_time[0] == x_dta['Week'])[0][0], np.where(tr_time[1] == x_dta['Week'])[0][0]
te_s_ind, te_e_ind = np.where(te_time[0] == x_dta['Week'])[0][0], np.where(te_time[1] == x_dta['Week'])[0][0]

target_col = [s for s in x_dta.columns.values if target_key == s]    
tmp_x_col = [s for s in x_dta.columns.values if 'Prev_'+target_key in s]
tmp_x_col1 = [s for s in x_dta.columns.values if 'Var_'+target_key in s]
week_col = [s for s in x_dta.columns.values if 'Week' in s]

tmp_x_set1, tmp_x_set2 = x_dta[tmp_x_col], x_dta[tmp_x_col1]

if target_key == 'Product':
    tmp_x_set1_re = tmp_x_set1.iloc[:,:col_k]
    tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
    x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set1_re, tmp_x_set2_re], axis=1)
    train_col = list(tmp_x_col[:col_k]+tmp_x_col1[:col_k1])    
elif target_key == 'Sale':
    tmp_x_set2_re = tmp_x_set2.iloc[:,:col_k1]
    x_dta_re = pd.concat([x_dta[list(week_col+target_col)], tmp_x_set2_re], axis=1)
    train_col = list(tmp_x_col1[:col_k1])


train_set = x_dta_re.iloc[s_ind:e_ind, :]
test_set = x_dta_re.iloc[te_s_ind:(te_e_ind+1), :]#.copy()

tr_x_set = train_set[train_col]
tr_y_set = np.array(train_set[target_col]).reshape(-1,1)+1e-5
te_x_set = test_set[train_col]
te_y_set = np.array(test_set[target_col]).reshape(-1,1)+1e-5



kernel =  theta1 * RBF(rbf_l) + WhiteKernel(w_noise)  + DotProduct(k_dot) ## RBF(2) + WhiteKernel(1)  + 
gp = GaussianProcessRegressor(kernel=kernel, alpha=g_alpha, n_restarts_optimizer=3)
#tr_x_ = np.atleast_2d([np.arange(0, np.shape(tr_x_set)[0])]).T
gp.fit(tr_x_set, tr_y_set)
train_pred, train_sigma = gp.predict(tr_x_set, return_std=True)
test_pred, test_sigma = gp.predict(te_x_set, return_std=True)
x_pred_ = np.concatenate((train_pred, test_pred)) 
y_ = np.concatenate((tr_y_set, te_y_set)) 

real_ = np.mean(np.ones_like(tr_y_set) - np.clip(np.abs(tr_y_set - train_pred) / tr_y_set , 0, 1)) * 100     
fcst_ = np.mean(np.ones_like(te_y_set) - np.clip(np.abs(te_y_set - test_pred) / te_y_set , 0, 1)) * 100    
print(real_, fcst_)

plt.figure(figsize=(20,10))
plt.plot(y_, color='blue', marker='o')
plt.plot(x_pred_, color='red', linestyle='dashed', marker='o')
plt.axvline(len(tr_y_set)-1, c='k')